In [17]:
import sklearn.feature_selection
from sklearn.model_selection import train_test_split
import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing

import numpy as np
import pandas as pd
import math
from pydataset import data

#### 1.) Load the tips dataset.


In [2]:
df = data('tips')
df.head(2)


,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3


##### Create a column named price_per_person. This should be the total bill divided by the party size.

In [3]:
df = df.rename(columns={'size':'party_size'})
df['price_per_person'] = df.total_bill / df.party_size

df.head(2)

,total_bill,tip,sex,smoker,day,time,party_size,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,3.446667


##### Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

 - Tip Percentage

In [4]:
df['tip_percentage'] = df.tip / df.total_bill

###### Use select k best to select the top 2 features for predicting tip amount. What are they?

In [5]:
# Pay attention to numeric features
df = df[['total_bill', 'tip', 'party_size', 
         'price_per_person','tip_percentage']]
df.head(2)

,total_bill,tip,party_size,price_per_person,tip_percentage
1,16.99,1.01,2,8.495000,0.059447
2,10.34,1.66,3,3.446667,0.160542


In [6]:
train_validate, test = train_test_split(df, 
                                        test_size=0.2, 
                                        random_state=123)

train, validate = train_test_split(train_validate, 
                                   test_size=0.3, 
                                   random_state=123)

train.shape, validate.shape, test.shape

((136, 5), (59, 5), (49, 5))

In [7]:
# Define the X and y's while dropping the target 
target = 'tip'
    
# split train into X (dataframe, drop target) & y (series, keep target only)
x_train = train.drop(columns=[target])
y_train = train[target]

# split validate into X (dataframe, drop target) & y (series, keep target only)
x_val = validate.drop(columns=[target])
y_val = validate[target]

# split test into X (dataframe, drop target) & y (series, keep target only)
x_test = test.drop(columns=[target])
y_test = test[target]


In [8]:
# Scale the numerical data to make it easier to compare
   
#Make the scaler
scaler = sklearn.preprocessing.MinMaxScaler()

#Fit the scaler
scaler.fit(x_train)

# Use the scaler
x_train_scaled = scaler.transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)
    

In [9]:
# Make the Model
kbest = sklearn.feature_selection.SelectKBest(
    sklearn.feature_selection.f_regression, 
    k=2)
# Fit the Model
kbest.fit(x_train_scaled, y_train)

# Use the model
kb_feats = x_train.columns[kbest.get_support()].tolist()
kb_feats

['total_bill', 'party_size']

##### Use recursive feature elimination to select the top 2 features for tip amount. What are they?


In [10]:
# Make the model
lm = sklearn.linear_model.LinearRegression(normalize = True)
rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=2)
# Fit the model
rfe.fit(x_train_scaled,y_train)

#Use the model
rfe_feats = x_train.columns[rfe.support_]
rfe_feats

Index(['total_bill', 'tip_percentage'], dtype='object')

##### Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

 - Because....

 - It does change. total_bill is included when increasing RFE size to 3

#### 2.) Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [14]:
def select_kbest(x, y, k):
    # Make the Model
    kbest = sklearn.feature_selection.SelectKBest(
        sklearn.feature_selection.f_regression, 
        k=k)
    
    # Fit the Model
    kbest.fit(x, y)
    
    # Use the model and make a list
    return x.columns[kbest.get_support()].tolist()


In [12]:
select_kbest(x_train_scaled, y_train, 2)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

#### 3.) Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [15]:
def rfe(x, y, k): 
    # Make the model
    lm = sklearn.linear_model.LinearRegression(normalize = True)
    rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=k)
    
    # Fit the model
    rfe.fit(x,y)
    
    #Use the model
    return x.columns[rfe.support_].tolist()


In [16]:
rfe(x_train, y_train, 2)


['party_size', 'tip_percentage']

#### 4.) Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [ ]:
df = data('swiss')


In [ ]:
df.head()


In [ ]:
train_validate, test = train_test_split(df, 
                                        test_size=.2, 
                                        random_state=123)

train, validate = train_test_split(train_validate, 
                                   test_size=.3, 
                                   random_state=123)

train.shape, validate.shape, test.shape


In [ ]:
target = 'Fertility'
# split train into X (dataframe, drop target) & y (series, keep target only)
x_train = train.drop(columns=[target])
y_train = train[target]

# split validate into X (dataframe, drop target) & y (series, keep target only)
x_val = validate.drop(columns=[target])
y_val = validate[target]

# split test into X (dataframe, drop target) & y (series, keep target only)
x_test = test.drop(columns=[target])
y_test = test[target]


In [ ]:
#Make the scaler
scaler = sklearn.preprocessing.MinMaxScaler()

#Fit the scaler
scaler.fit(x_train)

# Use the scaler
x_train_scaled = scaler.transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)


In [ ]:
x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns)
x_val_scaled = pd.DataFrame(x_val_scaled, columns=x_train.columns)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_train.columns)


In [ ]:
select_kbest(x_train, y_train, 3)


In [ ]:
rfe(x_train, y_train, 3)
